# Xây dựng với các Mô hình Mistral

## Giới thiệu

Bài học này sẽ bao gồm:
- Khám phá các Mô hình Mistral khác nhau
- Hiểu các trường hợp sử dụng và kịch bản cho từng mô hình
- Các mẫu mã minh họa các tính năng độc đáo của từng mô hình.


## Các Mô hình Mistral

Trong bài học này, chúng ta sẽ khám phá 3 mô hình Mistral khác nhau:  
**Mistral Large**, **Mistral Small** và **Mistral Nemo**.

Mỗi mô hình này đều có sẵn miễn phí trên thị trường Mô hình Github. Mã trong sổ tay này sẽ sử dụng các mô hình này để chạy mã. Dưới đây là thêm chi tiết về việc sử dụng Mô hình Github để [nguyên mẫu với các mô hình AI](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).


## Mistral Large 2 (2407)
Mistral Large 2 hiện là mô hình chủ lực từ Mistral và được thiết kế cho doanh nghiệp.

Mô hình là một bản nâng cấp so với Mistral Large gốc bằng cách cung cấp
- Cửa sổ ngữ cảnh lớn hơn - 128k so với 32k
- Hiệu suất tốt hơn trong các nhiệm vụ Toán học và Lập trình - độ chính xác trung bình 76,9% so với 60,4%
- Hiệu suất đa ngôn ngữ được cải thiện - các ngôn ngữ bao gồm: Tiếng Anh, Tiếng Pháp, Tiếng Đức, Tiếng Tây Ban Nha, Tiếng Ý, Tiếng Bồ Đào Nha, Tiếng Hà Lan, Tiếng Nga, Tiếng Trung, Tiếng Nhật, Tiếng Hàn, Tiếng Ả Rập và Tiếng Hindi.

Với những tính năng này, Mistral Large nổi bật trong
- *Tạo nội dung tăng cường truy xuất (RAG)* - nhờ cửa sổ ngữ cảnh lớn hơn
- *Gọi hàm* - mô hình này có gọi hàm gốc cho phép tích hợp với các công cụ và API bên ngoài. Các cuộc gọi này có thể được thực hiện song song hoặc lần lượt theo thứ tự tuần tự.
- *Tạo mã* - mô hình này xuất sắc trong việc tạo mã Python, Java, TypeScript và C++.


### Ví dụ RAG sử dụng Mistral Large 2


Trong ví dụ này, chúng tôi đang sử dụng Mistral Large 2 để chạy một mẫu RAG trên một tài liệu văn bản. Câu hỏi được viết bằng tiếng Hàn và hỏi về các hoạt động của tác giả trước khi vào đại học.

Nó sử dụng Mô hình Nhúng Cohere để tạo các nhúng của tài liệu văn bản cũng như câu hỏi. Đối với mẫu này, nó sử dụng gói Python faiss làm kho vector.

Lời nhắc gửi đến mô hình Mistral bao gồm cả câu hỏi và các đoạn được truy xuất có độ tương đồng với câu hỏi. Mô hình sau đó cung cấp câu trả lời bằng ngôn ngữ tự nhiên.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small là một mô hình khác trong gia đình Mistral thuộc loại premier/enterprise. Như tên gọi, mô hình này là một Mô hình Ngôn ngữ Nhỏ (SLM). Những lợi thế khi sử dụng Mistral Small là: 
- Tiết kiệm chi phí so với các Mistral LLM như Mistral Large và NeMo - giảm giá 80%
- Độ trễ thấp - phản hồi nhanh hơn so với các LLM của Mistral
- Linh hoạt - có thể triển khai trên nhiều môi trường khác nhau với ít hạn chế về tài nguyên cần thiết. 


Mistral Small rất phù hợp cho: 
- Các tác vụ dựa trên văn bản như tóm tắt, phân tích cảm xúc và dịch thuật. 
- Các ứng dụng có tần suất yêu cầu cao nhờ hiệu quả về chi phí 
- Các tác vụ mã có độ trễ thấp như xem xét và gợi ý mã nguồn 


## So sánh Mistral Small và Mistral Large

Để hiển thị sự khác biệt về độ trễ giữa Mistral Small và Large, hãy chạy các ô bên dưới.

Bạn sẽ thấy sự khác biệt về thời gian phản hồi trong khoảng 3-5 giây. Cũng lưu ý độ dài và phong cách phản hồi trên cùng một lời nhắc.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

So với hai mô hình khác được thảo luận trong bài học này, Mistral NeMo là mô hình duy nhất miễn phí với Giấy phép Apache2.

Nó được xem như một bản nâng cấp của LLM mã nguồn mở trước đó từ Mistral, Mistral 7B.

Một số tính năng khác của mô hình NeMo là:

- *Phân tách token hiệu quả hơn:* Mô hình này sử dụng bộ phân tách Tekken thay vì tiktoken được sử dụng phổ biến hơn. Điều này cho phép hiệu suất tốt hơn trên nhiều ngôn ngữ và mã nguồn.

- *Tinh chỉnh:* Mô hình cơ sở có sẵn để tinh chỉnh. Điều này cho phép linh hoạt hơn cho các trường hợp sử dụng cần tinh chỉnh.

- *Gọi hàm gốc* - Giống như Mistral Large, mô hình này đã được huấn luyện trên gọi hàm. Điều này làm cho nó trở nên độc đáo khi là một trong những mô hình mã nguồn mở đầu tiên làm được điều này.


## Mistral NeMo

So với hai mô hình khác được thảo luận trong bài học này, Mistral NeMo là mô hình duy nhất miễn phí với giấy phép Apache2.

Nó được xem như một bản nâng cấp của LLM mã nguồn mở trước đó từ Mistral, Mistral 7B.

Một số tính năng khác của mô hình NeMo bao gồm:

- *Phân tách token hiệu quả hơn:* Mô hình này sử dụng bộ phân tách Tekken thay vì tiktoken phổ biến hơn. Điều này cho phép hiệu suất tốt hơn trên nhiều ngôn ngữ và mã nguồn.

- *Tinh chỉnh:* Mô hình cơ sở có sẵn để tinh chỉnh. Điều này cho phép linh hoạt hơn cho các trường hợp sử dụng cần tinh chỉnh.

- *Gọi hàm gốc* - Giống như Mistral Large, mô hình này đã được huấn luyện trên gọi hàm. Điều này làm cho nó trở nên độc đáo khi là một trong những mô hình mã nguồn mở đầu tiên làm được điều này.


### So sánh các bộ tách từ 

Trong ví dụ này, chúng ta sẽ xem cách Mistral NeMo xử lý việc tách từ so với Mistral Large. 

Cả hai ví dụ đều sử dụng cùng một lời nhắc nhưng bạn sẽ thấy rằng NeMo trả về ít token hơn so với Mistral Large. 


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## Việc học không dừng lại ở đây, hãy tiếp tục hành trình

Sau khi hoàn thành bài học này, hãy xem bộ sưu tập [Học tập AI Tạo sinh của chúng tôi](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) để tiếp tục nâng cao kiến thức về AI Tạo sinh của bạn!


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Tuyên bố từ chối trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ gốc của nó nên được coi là nguồn tham khảo chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp do con người thực hiện. Chúng tôi không chịu trách nhiệm về bất kỳ sự hiểu lầm hoặc giải thích sai nào phát sinh từ việc sử dụng bản dịch này.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
